In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install -q ultralytics mlflow opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 8.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 58.2 kB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 6.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 2.4 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.3 which is incompatible.
mkl-u

In [2]:
!pip install -U "numpy<2.0" "mlflow>=2.15.0" 


  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.3 which is incompatible.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.40.3 which is incompatible.
goog

In [3]:
from ultralytics import YOLO, settings
import os, time, pathlib
import mlflow
import mlflow.pytorch


In [4]:
MLFLOW_DIR = "kaggle/working/mlruns"
os.makedirs(MLFLOW_DIR, exist_ok = True)
mlflow.set_tracking_uri(f"file:{MLFLOW_DIR}")
settings.update({"mlflow": True})

In [5]:
DATA_DIR = "/kaggle/input/vehicledetection/VehiclesDetectionDataset"

In [13]:
MODEL = "yolov8n.pt"
EPOCHS = 16
IMGSZ = 640
BATCH = 16
DEVICE = "0,1"
WORKERS = 4
SAVE_DIR = "/kaggle/working/runs"



In [15]:
mlflow.set_experiment("yolo-vehiclesdetection")
with mlflow.start_run(run_name="yolov8n_ob"):
    mlflow.pytorch.autolog()
    model= YOLO(MODEL)

    results = model.train(
        data = "/kaggle/input/vehicledetection/VehiclesDetectionDataset/dataset.yaml",
        epochs = EPOCHS,
        imgsz = IMGSZ,
        batch = BATCH,
        device = DEVICE,
        workers = WORKERS,
        project = SAVE_DIR,
        name = "yolov8n-vehicles",
        pretrained = True,
        amp= True
    )
    
        
    train_time = time.time()
    mlflow.log_metric("train_time_sec", train_time)

    last_ckpt = next(pathlib.Path(SAVE_DIR).rglob("**/weights/last*.pt"),None)
    if last_ckpt:
        mlflow.log_artifact(str(last_ckpt), artifact_path="checkpoints")



Ultralytics 8.3.223 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/vehicledetection/VehiclesDetectionDataset/dataset.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=16, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n-vehicles

2025/11/03 16:51:25 WARNING mlflow.spark: With Pyspark >= 3.2, PYSPARK_PIN_THREAD environment variable must be set to false for Spark datasource autologging to work.
2025/11/03 16:51:25 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.


       1/16      1.11G      1.025      2.978      1.409         19        640: 100% ━━━━━━━━━━━━ 55/55 4.6it/s 11.9s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 6.3it/s 1.3s0.2s
                   all        250        454      0.643     0.0772      0.329      0.221

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/16      1.36G      1.137      2.356      1.458         19        640: 100% ━━━━━━━━━━━━ 55/55 5.7it/s 9.7s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 6.2it/s 1.3s0.2s
                   all        250        454       0.33      0.417      0.321       0.17

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       3/16      1.38G      1.174      2.329       1.48         17        640: 100% ━━━━━━━━━━━━ 55/55 5.8it/s 9.5s0.2s
                 Class     Images  Instances 

In [16]:
VIDEO_IN = "/kaggle/input/vehicledetection/TestVideo/TrafficPolice.mp4"
model = YOLO(str(last_ckpt))
pred = model.predict(source=VIDEO_IN, imgsz=IMGSZ, save=True, show=False)
pred_path = next(pathlib.Path("/kaggle/working/runs/predict").rglob("*.mp4"), None)
if pred_path:
    mlflow.log_artifact(str(pred_path), artifact_path="inference_videos")



WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/2362) /kaggle/input/vehicledetection/TestVideo/TrafficPolice.mp4: 384x640 7 Cars, 40.1ms
video 1/1 (frame 2/2362) /kaggle/input/vehicledetection/TestVideo/TrafficPolice.mp4: 384x640 7 Cars, 7.1ms
video 1/1 (frame 3/2362) /kaggle/input/vehicledetection/TestVideo/TrafficPolice.mp4: 384x640 7 Cars, 7.5ms
video 1/1 (frame 4/2362) /kaggle/input/vehicledetection/TestVideo/TrafficPolice.mp4: 384x640 7 Cars, 8.2ms
video 1/1 (frame 5/2362) /k